In [ ]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from imgaug import augmenters as iaa
import numpy as np
print(tf.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
%cd /content/gdrive/My Drive/Kaggle

In [ ]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

In [ ]:
import zipfile
import shutil

shutil.rmtree('/content/gdrive/MyDrive/Kaggle/face_data')
zip_loc = '/content/gdrive/MyDrive/Kaggle/face-expression-recognition-dataset.zip'
data_loc = '/content/gdrive/MyDrive/Kaggle/face_data'
data_extract = zipfile.ZipFile(zip_loc, 'r')
data_extract.extractall(data_loc)
data_extract.close()

shutil.rmtree('/content/gdrive/MyDrive/Kaggle/face_data/images/images')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_dir = '/content/gdrive/MyDrive/Kaggle/face_data/images/train'
valid_dir = '/content/gdrive/MyDrive/Kaggle/face_data/images/validation'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    horizontal_flip=True,
    zca_whitening=True,
    brightness_range=(0.5, 1.5),
    zoom_range=(0.5, 1.5),
    shear_range=0.3,
    fill_mode='nearest'
    )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=256,
    class_mode='categorical',
)

valid_generator = train_datagen.flow_from_directory(
    valid_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=256,
    class_mode='categorical',
)

In [ ]:
x_batch, y_batch = next(train_generator)
x= train_generator.next()

plt.figure(figsize=(25,25)) # specifying the overall grid size

for i in range(25):
    plt.subplot(5,5,i+1)    # the number of images in the grid is 5*5 (25)
    image = x_batch[i]
    plt.imshow((tf.squeeze(image)))
    #plt.imshow(image)

plt.show()

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.9 and logs.get('val_accuracy') > 0.9):
            print("\nAccuracy is Optimal")
            self.model.stop_training = True

In [ ]:
callbacks = myCallback()

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(7, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='Nadam',
    metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=50,
    validation_data=valid_generator,
    validation_steps=5,
    verbose=1,
    callbacks=[callbacks]
)

In [ ]:
from tensorflow.keras.models import model_from_json
import numpy

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("weights.h5")
print("Saved model to disk")